In [1]:

import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import math
from scipy.optimize import curve_fit
import glob
import os
import matplotlib.backends.backend_pdf

In [2]:
#print glob.glob("/*.fits") # This only works for Unix

file_name = []

for file in os.listdir("./"):
    if file.endswith(".fits"):
        file_name.append(file)


In [3]:
f = open('line_list.dat', 'r')
g = open('Instrument_list.dat','r')
h = open('speclist_extra.dat', 'r')


#delta_v = 1000.0;
c = 300000.0;
line_name = []
line_lam = []
delta_lam = []
lambda_over_c = []
line_num = 0

# Getting wavelengths and names of the important lines
for line in f.readlines():
    row = line.strip()
    columns = row.split()
    line_lam.append(float(columns[0]))
    line_name.append(columns[1]+columns[2])
    #lambda_over_c.append(float(columns[0])/c) # delta_lambda = lambda*(delta_v)/c
    #print (delta_v*float(columns[0])/c)
    line_num = line_num + 1

f.close()
line_wavelength = np.array(line_lam)


In [4]:
instrument = []
resolution = []
plot_window = []

for line in open('Instrument_list.dat'):
    row = line.strip()
    if not row.startswith("#"):
        columns = row.split()
        instrument.append(columns[0] + ' ' + columns[1])
        resolution.append(float(columns[2]))
        plot_window.append(float(columns[3]))
        
        
grb_num = 0;
grb_file_name = []
grb_redshift = []
grb_instrument = []
#for line in open('speclist.dat'):
#for i in range (0, 65):
#    h.readline()
    
pdf = matplotlib.backends.backend_pdf.PdfPages("GRB_Plots_extra.pdf")

    
for line in h.readlines():
    row=line.strip()
    if not row.startswith("#"):
        columns = line.split()
        grb_file_name.append(columns[0])
        grb_redshift.append(float(columns[1]))
        grb_instrument.append(columns[2] + ' ' + columns[3])
        
        print(columns[0])
        
        hdulist = fits.open(grb_file_name[grb_num])
        Flux = hdulist[0]
        Error = hdulist[1]
        Wavelength = hdulist[2]
        
        redshift = grb_redshift[grb_num] #Getting the GRB Redshift
        Wavelength_rest = np.array(Wavelength.data/(1+redshift)) # Transforming to rest wavelength
        Flux_norm = np.array(Flux.data)
        Error_norm = np.array(Error.data)
        
        instru_num = instrument.index(grb_instrument[grb_num]) #Finding the index of the array corresponding to the particular instrument used
        instru_res = resolution[instru_num] # Getting Instrument resolution, if it is needed in future
        delta_v = plot_window[instru_num]  # Getting the corresponding plot window as written in Instrument_list.dat 
        
        delta_lam = delta_v*line_wavelength/c # delta_lambda = lambda*(delta_v)/c
        delta_lambda = np.array(delta_lam)
        
        #Plotting the lines which are present in the data subplot of 'nrows' rows and 'ncols' columns
        Wavelength_box = []
        Velocity_box = []
        Flux_box = []
        Error_box = []

        ncols = 4
        nrows = int(math.ceil(float(line_num)/ncols))
        f, axarr = plt.subplots(nrows, ncols)
        
        plot_name = grb_file_name[grb_num]
        plot_name = plot_name[:-5]
        plt.suptitle((plot_name +' z = '+str(redshift) + ' ' + str(grb_instrument[grb_num]) ) , fontsize = 30)
        

        for i in range(0,line_num):
            Wavelength_window = Wavelength_rest[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
            Velocity_window = ((Wavelength_window - line_wavelength[i])*c)/line_wavelength[i]
            Flux_window = Flux_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
            Error_window = Error_norm[(Wavelength_rest >= (line_wavelength[i]-delta_lambda[i])) * (Wavelength_rest <= (line_wavelength[i]+delta_lambda[i]))]
            
            Wavelength_box.append(Wavelength_window)
            Velocity_box.append(Velocity_window)
            Flux_box.append(Flux_window)
            Error_box.append(Error_window)
    
            rownum = int(i%nrows)
            colnum = int(i//nrows)
            axarr[rownum, colnum].errorbar(Velocity_window, Flux_window, yerr=Error_window, ecolor='r')
            axarr[rownum, colnum].set_ylim(-1.5, 1.5) #axarr[rownum, colnum].set_ylim(-0.1, 1.5)
            axarr[rownum, colnum].text(0, 1.3, line_name[i], fontsize=20)
            #axarr[rownum, colnum].set_xlabel('Velocity (km/s)', fontsize=15)
            #axarr[rownum, colnum].set_ylabel('Normalized Flux (arbitrary units)', fontsize=15)
            max_yticks = 5
            max_xticks = 5            
            yloc = plt.MaxNLocator(max_yticks)
            xloc = plt.MaxNLocator(max_xticks)
            axarr[rownum, colnum].yaxis.set_major_locator(yloc)
            axarr[rownum, colnum].xaxis.set_major_locator(xloc)
            axarr[rownum, colnum].tick_params(axis='x', labelsize=15)
            axarr[rownum, colnum].tick_params(axis='y', labelsize=15)

            axarr[rownum, colnum].axhline(y=1, color='k', ls='dashed')
            
        figure = plt.gcf() # get current figure
        figure.set_size_inches(15, 15)
        # when saving, specify the DPI
        #plt.savefig(plot_name, dpi = 150, bbox_inches='tight')
        pdf.savefig(figure, dpi = 150, bbox_inches='tight')
        # This is the final step: To increment!! Keep this at the end all the time, otherwise you'll get errors!!
        plt.close()
        grb_num = grb_num+1
pdf.close()

011211.fits
020813_uvreb.fits
021004_uvreb.fits
050408.fits
050709.fits
050730_uves.fits
050730_uvreb.fits
050820A_HIRES.fits
050820A_uves.fits
050824.fits
050908_deimos.fits
050922C_FORS.fits
050922C_uves.fits
051221A.fits
060123.fits
060124_ESI.fits
060124_LRIS.fits
060206_KAST.fits
060418.fits
060418_uves.fits
060502A.fits
060526_sp1.fits
060604.fits
060607A_uves.fits
60708A.fits
060912A.fits
070125.fits
070125fors.fits
070208.fits
070318.fits
070612A.fits
070724A.fits
071025b.fits
071025r.fits
071025b.fits
071031_FORS.fits
071031_uves.fits
080210_FORS_sp1.fits
080210_FORS2_1400V.fits
080310_uves.fits
080310_uves_old.fits
080413A_Gemini.fits
080607A_LRIS1200.fits
080607A_LRISB600.fits
080607A_LRISR400.fits
080804_gemini.fits
080804_uves.fits
080804_uves4B.fits
080804_uves4Blue.fits
080804_uves5R.fits
080805.fits
080810_HIRES.fits
080810_NOT.fits
080905B.fits
081007.fits
081008.fits
081008_gemini.fits
081008A_uvesB.fits
081008A_uvesR.fits
081029.fits
090424.fits
090424A.fits
090426A.

In [ ]:
pdf.close()